In [ ]:

import json
import requests
from bs4 import BeautifulSoup as bs
import datetime
import time

s=requests.Session()
s.keep_alive = False
headers={'Host':'www.bjguahao.gov.cn',
'Upgrade-Insecure-Requests':'1',
'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

kk=[]
def loginbj():
    url='http://www.bjguahao.gov.cn/quicklogin.htm'
    data={'mobileNo':'13915729766',
            'password':'xue11wei',
            'yzm':'',
            'isAjax':'true'}
    res=s.post(url,headers=headers,data=data)
    resp=res.text     #这个返回赋值的是ｓｔｒ格式，然后下面语句就是转化成ｊｓｏｎ，字典格式
    jresp=json.loads(resp)
    if jresp['msg']=='OK':
  
        return True
    else:
        return False

def recheck():
    if loginbj()==True:
        url='http://www.bjguahao.gov.cn/islogin.htm'
        data={'isAjax':'true'}
        res=s.post(url,data=data)
        jresp=json.loads(res.text)
        print('{},您登录成功了'.format(jresp['username']))
        return True
    else:
        print('显然没登录成功，再试试')
        return False
    
def guahao():
    if recheck()==True:
        sortdata()
       
        
        
def sendorder(): #发送手机接收码
    url='http://www.bjguahao.gov.cn/v/sendorder.htm'
    res=s.post(url).text
    jres=json.loads(res)

    if jres['msg']=='OK.':
        print('稍等片刻，等着输入手机接收到的验证码')
    else:
        print('验证码没有成功发送哎,'+jres['msg'])

def shuruma():
    code=input('请输入手机接收到的码：')
    return code

def book():
    hospitalId=input("请输入你想挂号的医院ID，3位码：")
    departmentId=input("请输入想挂的科室ID，9位编码")
    url='http://www.bjguahao.gov.cn/dpt/appoint/'+hospitalId+'-'+departmentId+'.htm'
    res=s.get(url)
    res.encoding='gzip'
    soup=bs(res.text,'html.parser')
    for sy in soup.select('td.ksorder_kyy input'):
        k=sy['value']
        kk.append(k)
    return kk,hospitalId,departmentId

def sortdata():
    validtime,hospitalId,departmentId=book()
    for i in range(0,len(validtime)):
        dutyCode=validtime[i].split('_')[1].split('_')[0]  #上午还是下午
        dutyDate=validtime[i].split('_')[2]
        url='http://www.bjguahao.gov.cn/dpt/partduty.htm'
        data1={'hospitalId':hospitalId,
            'departmentId':departmentId,
            'dutyCode':dutyCode,
            'dutyDate':dutyDate,
            'isAjax':'true'}
        res=s.post(url,data=data1).text
        res=json.loads(res)   #把返回的变成ｊｓｏｎ格式
        data=res['data']
        docname=data[0]['doctorName']
        dutySourceId=data[0]['dutySourceId']
        hospitalId=data[0]['hospitalId']
        departmentId=data[0]['departmentId']
        doctorId=data[0]['doctorId']
        url1='http://www.bjguahao.gov.cn/order/confirm/'+str(hospitalId)+'-'+str(departmentId)+'-'+str(doctorId)+'-'+str(dutySourceId)+'.htm'
        res=s.get(url1,headers=headers)
        res.encoding='gzip'
        soup=bs(res.text,'html.parser')
        for id in soup.select('div.Rese_db dl.Rese_db_dl dd p input'):
            patientid=id['value']
        print('{},是你的就诊医生'.format(docname))
        print('你的就诊日期是{}'.format(dutyDate)+'上午为1，下午为2--%d--'%(dutyCode))
        sendorder()
        smsverifycode=shuruma()
      
        querystring={
                    'dutySourceId':dutySourceId,
                    'hospitalId':hospitalId,
                    'departmentId':departmentId,
                    'doctorId':doctorId,
                    'patientId':patientid,  #这个要用ｂｅａｕｔｉｆｕｌｓｏｕｐ解析处理
                    'hospitalCardId':'',
                    'medicareCardId':'',
                    'reimbursementType':'5',
                    'smsVerifyCode':smsverifycode,
                    'childrenBirthday':'',
                    'isAjax':'true'}
        url2='http://www.bjguahao.gov.cn/order/confirm.htm'
        res1=s.post(url2,data=querystring,headers=headers)
        jres1=json.loads(res1.text)
        print(jres1['msg'])
        if jres1['msg'] is not None:
            print("抢成功了，退出")
            exit()
        else:
            continue

guahao()

In [ ]:
for i in range(10):
    print('----%d-----'%i)
    for j in range(10):
        if j>5:
            break
        print(j)
        

In [ ]:
for i in range(10):
    print('----%d----'%i)
    for j in range(10):
        if j>5 and j<=8:
            print('我是谁')
            exit()
        print(j)
    

In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
import json
import requests
from bs4 import BeautifulSoup as bs
import datetime
import time

s=requests.Session()
s.keep_alive = False
headers={'Host':'www.bjguahao.gov.cn',
'Upgrade-Insecure-Requests':'1',
'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

hpId=[]
hpname=[]
hpurl=[]
hpdepartId=[]
departname=[]
def aquirehospitalId():
    url='http://www.bjguahao.gov.cn/hp.htm'
    re=s.get(url,headers=headers)
    re.encoding='utf-8'
    soup=bs(re.text,'lxml')
    page=soup.select('ul.yiyuan_list_rq_ul li')
    stringpage=page[10].get_text()
    page=stringpage.split('共')[1].split('页')[0]
    page=int(page)
    for i in range(0,page+1):
        ul='http://www.bjguahao.gov.cn/hp/'+str(i)+',0,0,0.htm?'
     
        req=s.get(ul)
        sp=bs(req.text,'lxml')
        for name in sp.select('p.yiyuan_co_titl a'):
            hpurl.append('http://www.bjguahao.gov.cn'+name['href'])
            name=str(name)
            hospitalId=name.split('<a href="/hp/appoint/1/')[1].split('.')[0]
            hpId.append(hospitalId)
        for title in soup.select('p.yiyuan_co_titl a'):  
            hpname.append(title.get_text())
    k={'医院名称':hpname}
    kk={'医院的id码':hpId}
    return k,kk,hpurl  #返回对应的医院字典



def aquiredepartmentId():
    yy,yyid,hpurl=aquirehospitalId()
    for url in hpurl:
        req=s.get(url,headers=headers)
        sp=bs(req.text,'lxml')
        for deptId in sp.select('ul li a.kfyuks_islogin'):
            departname.append(deptId.get_text())
            deptId=str(deptId)
            deptId=deptId.split('<a class="kfyuks_islogin" href="/dpt/appoint/')[1].split('.')[0]
            hpdepartId.append(deptId)
    departn={"部门名称":departname}
    hpdepid={"部门Id":hpdepartId}
    return yy,yyid,departn,hpdepid

def transfer2frame(a,b,c,d):
    fra=DataFrame(a)
    frb=DataFrame(b)
    frc=DataFrame(c)
    frd=DataFrame(d)
    return fra,frb,frc,frd

n,m,k,x=aquiredepartmentId() 
e,f,g,h=transfer2frame(n,m,k,x)

df=pd.concat([e,f],axis=1)

df.to_csv('bjgh.csv',index=0)
dff=pd.concat([g,h],axis=1)
dff.to_csv('bjghbm.csv',index=0)

In [ ]:
string='<a href="/hp/appoint/1/1.htm">北京协和医院</a>'
k=string.split('<a href="/hp/appoint/1/')[1].split('.')[0]
print(k)

In [ ]:
k='http://www.bjguahao.gov.cn/hp/2,0,0,0.htm?'

In [ ]:
c=[1,2,3,4,5,6]
cc={"c":c}
print(cc)